In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

import math

In [2]:
data=pd.read_csv('~\Downloads\daily_data.csv')
data

,datetime,high,low,close,open,volume
0,2018-01-01,13818.55,12750.00,13135.00,13715.65,6970.441076
1,2018-01-02,14750.00,12890.02,14580.00,13135.00,16359.347340
2,2018-01-03,15473.49,14150.00,14749.97,14576.87,19457.238601
3,2018-01-04,15280.00,13918.04,14811.00,14710.00,18783.876561
4,2018-01-05,17118.13,14600.00,16579.00,14805.01,22383.657333
...,...,...,...,...,...,...
1468,2022-01-08,42246.37,40501.00,40679.82,41707.04,27031.900870
1469,2022-01-09,42300.00,40520.00,41966.99,40679.82,27291.009660
1470,2022-01-10,42786.70,39650.00,41604.58,41966.99,51064.904360
1471,2022-01-11,43100.00,41158.99,42755.00,41604.58,38062.818280


In [9]:
SMA_100=[]
for i in range(100):
    SMA_100.append(data.close[i])
for i in range(100,len(data.close)):
    temp=0
    for j in range(i-100,i):
        temp+=data.close[j]
    temp=temp/100
    SMA_100.append(temp)


SMA_20=[]
for i in range(100):
    SMA_20.append(data.close[i])
for i in range(100,len(data.close)):
    temp=0
    for j in range(i-20,i):
        temp+=data.close[j]
    temp=temp/20
    SMA_20.append(temp)

def EMA(stock):
    EMA=[]
    for i in range(100):
        EMA.append(0)
    EMA.append(data.close[100])
    for i in range(101,len(data.close)):
        temp=((data.close[i]*(2))/11)+(EMA[i-1])*(1-(2/11))
        EMA.append(temp)
    stock['EMA']=EMA
EMA(data)
EMA=data['EMA']
data['SMA100']=SMA_100
data['SMA20']=SMA_20

In [10]:
signal=[]
datetime=[]
for i in range(len(data.datetime)):
    datetime.append(data.datetime[i])
    if(i<100):
        signal.append(0)
        continue
    SMA_100_diff=SMA_100[i]-SMA_100[i-1]
    EMA_diff=EMA[i]-EMA[i-1]
    ratio=EMA_diff/SMA_100_diff
    if(ratio>1):
        signal.append(1)
    elif(ratio<-1):
        signal.append(-1)
    else:
        signal.append(0)
a=pd.DataFrame(columns=['datetime','signal'])
a['datetime']=datetime
a['signal']=signal
a
    

,datetime,signal
0,2018-01-01,0
1,2018-01-02,0
2,2018-01-03,0
3,2018-01-04,0
4,2018-01-05,0
...,...,...
1468,2022-01-08,-1
1469,2022-01-09,1
1470,2022-01-10,1
1471,2022-01-11,1


In [14]:
def to_submit(data,output):
    datetime=[]
    trade=[]
    signal=[]
    price=[]
    current=0
    for i in range(len(data.close)):
        if(i==(len(data.close)-1)):
            if(current==1):
                datetime.append(output.datetime[i])
#                 price.append(data.close[i])
                trade.append('close')
                signal.append(-1)
            elif(current==-1):
                datetime.append(output.datetime[i])
#                 price.append(data.close[i])
                trade.append('close')
                signal.append(1)
            else:
                break
            break
        if(output.signal[i]==1):
            if(current==1):
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(0)
            elif(current==0):
                datetime.append(output.datetime[i])
#                 price.append(data.close[i])
                trade.append('close')
                signal.append(1)
                current=1
            else:
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(1)
                current=0
            
        elif(output.signal[i]==-1):
            if(current==1):
                datetime.append(output.datetime[i])
                signal.append(-1)
#                 price.append(data.close[i])
                trade.append('close')
                current=0
            elif(current==0):
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(-1)
                current=-1
            else:
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(0)
                current=-1
        
        else:
            datetime.append(output.datetime[i])
            trade.append('close')
#             price.append(data.close[i])
            signal.append(0)
        
            
            
    to_submit=pd.DataFrame(columns=['datetime','signal','close\open'])
    to_submit['datetime']=datetime
    to_submit['signal']=signal
    to_submit['close\open']=trade
#     to_submit['price']=price
    return to_submit
            
            
def metrics(data,submit):
    capital=1000
    current=0
    temp=0
    portfolio_value=[]
    datetime=[]
    quantity=[]
    
    sharpe=[]
    returns=[]
    risk_free_rate=0.05       
    for i in range(len(data.close)):
        datetime.append(data.datetime[i])
        if(submit.signal[i]==1):
            if(current==0):
                portfolio_value.append(capital)
                returns.append((capital-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
                
                temp=capital/(data.close[i])
                quantity.append(temp)
                capital=0
                current=1
            else:
                capital=capital-(temp*data.close[i])
                returns.append((capital-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
                quantity.append(temp)
                portfolio_value.append(capital)
                temp=0
                current=0
                
        elif(submit.signal[i]==-1):
            if(current==0):
                portfolio_value.append(capital)
                temp=capital/(data.close[i])
                quantity.append(temp)
                returns.append((capital-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
                capital=capital+(temp*data.close[i])
                
                current=-1
            else:
                capital=capital+(temp*data.close[i])
                quantity.append(temp)
                current=0
                temp=0
                portfolio_value.append(capital)
                returns.append((capital-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
        
        else:
            if(current==0):
                portfolio_value.append(capital)
                quantity.append(temp)
                returns.append((capital-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
            elif(current==1):
                portfolio_value.append(capital+(temp*data.close[i]))
                quantity.append(temp)
                returns.append((capital+(temp*data.close[i])-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
            else:
                portfolio_value.append(capital-(temp*data.close[i]))
                quantity.append(temp)
                returns.append((capital-(temp*data.close[i])-1000)/10)
                sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
                
    a=pd.DataFrame(columns=['datetime','portfolio value'])
    a['datetime']=datetime
    a['portfolio value']=portfolio_value
#     a['sharpe']=sharpe
    a['quantity']=quantity
    
    a['daily_return'] = a['portfolio value'].pct_change()
    daily_volatility = a['daily_return'].std()
    annual_volatility = daily_volatility * math.sqrt(252)
    risk_free_rate = 0.03
    
    a['excess_return'] = a['daily_return'] - risk_free_rate / 252
    cumulative_excess_return = a['excess_return'].sum()
    negative_returns = [r for r in a['excess_return'] if r < 0]
    cumulative_volatility = a['daily_return'].std() * math.sqrt(252)
    hello=np.std(negative_returns)*math.sqrt(252)
    sharpe_ratio = cumulative_excess_return / cumulative_volatility
    sortino_ratio=cumulative_excess_return / hello
    
    
    
    start=[]
    end=[]
    start_value=[]
    end_value=[]
    duration=[]
    P_and_L=[]
    return_for_every_trade=[]
    pointer=0
    drawdown=[]
    for i in range(len(submit.signal)):
        if(submit.signal[i]==1):
            if(pointer==0):
                start.append(i)
                start_value.append(portfolio_value[i])
                pointer=1
            else:
                end.append(i)
                end_value.append(portfolio_value[i])
                return_for_every_trade.append(100*((end_value[-1]/start_value[-1])-1))
                P_and_L.append(end_value[-1]-start_value[-1])
                duration.append(end[-1]-start[-1])
                pointer=0
                minimum=np.min(portfolio_value[start[-1]:end[-1]])
                maximum=np.max(portfolio_value[start[-1]:end[-1]])
                drawdown.append(100*((maximum-minimum)/maximum))
#                 drawdown.append(100*((portfolio_value[start[-1]]/minimum)-1))
                
        elif(submit.signal[i]==-1):
            if(pointer==0):
                start.append(i)
                start_value.append(portfolio_value[i])
                pointer=-1
            else:
                end.append(i)
                end_value.append(portfolio_value[i])
                return_for_every_trade.append(100*((end_value[-1]/start_value[-1])-1))
                P_and_L.append(end_value[-1]-start_value[-1])
                duration.append(end[-1]-start[-1])
                pointer=0
                minimum=np.min(portfolio_value[start[-1]:end[-1]])
                maximum=np.max(portfolio_value[start[-1]:end[-1]])
                drawdown.append(100*((maximum-minimum)/maximum))
                
    b=pd.DataFrame(columns=['start','end'])
    b['start']=start
    b['end']=end
    b['duration']=duration
    b['return for every trade']=return_for_every_trade
    b['P and L']=P_and_L
    b['drawdown']=drawdown
    
    

    gross_profit=[]
    gross_loss=[]
    for i in range(len(P_and_L)):
        if(P_and_L[i]>0):
            gross_profit.append(P_and_L[i])
        else:
            gross_loss.append(P_and_L[i])
    print("GROSS PROFIT: ",np.sum(gross_profit))
    print("GROSS LOSS: ",np.sum(gross_loss))
    print("NET P and L: ",np.sum(P_and_L))
    print("TOTAL CLOSED TRADES: ",len(start))
    print("WIN RATE: ",100*(len(gross_profit)/(len(gross_profit)+len(gross_loss))))
    print("AVG WINNING TRADE: ",np.mean(gross_profit))
    print("AVG LOSING TRADE: ",np.mean(gross_loss))
    print("Largest Losing trade: ",min(gross_loss))
    print("Largest winning trade: ",max(gross_profit))
    print("Avg holding duration: ",np.mean(duration))
    
   
    
    print("Sharpe Ratio: ",sharpe_ratio)


#     std_dev_negative = np.std(negative_returns)
            
    print("Sortino Ratio: ",sortino_ratio)
    print("RETURNS: ",((np.sum(P_and_L))/10))
    print("MAX Drawdown: (in %) ",np.max(drawdown))
    
    return a,b
                

In [15]:
c,d=metrics(data,to_submit(data,a))

C:\Users\tanus\AppData\Local\Temp\ipykernel_22908\476992568.py:133: RuntimeWarning: divide by zero encountered in double_scalars
  sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))
C:\Users\tanus\AppData\Local\Temp\ipykernel_22908\476992568.py:94: RuntimeWarning: divide by zero encountered in double_scalars
  sharpe.append((np.mean(returns)-risk_free_rate)/(np.std(returns)))


GROSS PROFIT:  7920.842112349724
GROSS LOSS:  -6896.429667319436
NET P and L:  1024.4124450302875
TOTAL CLOSED TRADES:  131
WIN RATE:  45.80152671755725
AVG WINNING TRADE:  132.01403520582872
AVG LOSING TRADE:  -97.1328122157667
Largest Losing trade:  -1338.1828276564947
Largest winning trade:  1251.3488526649382
Avg holding duration:  9.068702290076336
Sharpe Ratio:  2.5644815993433947
Sortino Ratio:  3.5396362596798845
RETURNS:  102.44124450302874
MAX Drawdown: (in %)  54.05126668900062


In [16]:
c

,datetime,portfolio value,quantity,daily_return,excess_return
0,2018-01-01,1000.000000,0.000000,NaN,NaN
1,2018-01-02,1000.000000,0.000000,0.000000,-0.000119
2,2018-01-03,1000.000000,0.000000,0.000000,-0.000119
3,2018-01-04,1000.000000,0.000000,0.000000,-0.000119
4,2018-01-05,1000.000000,0.000000,0.000000,-0.000119
...,...,...,...,...,...
1468,2022-01-08,2021.479470,0.037171,0.019264,0.019145
1469,2022-01-09,1973.633967,0.037171,-0.023669,-0.023788
1470,2022-01-10,1973.633967,0.047438,0.000000,-0.000119
1471,2022-01-11,2028.207477,0.047438,0.027651,0.027532


In [13]:
d

,start,end,duration,return for every trade,P and L,drawdown
0,100,101,1,11.451719,114.517193,0.000000
1,102,127,25,-13.540055,-150.906241,23.443190
2,129,135,6,-9.457831,-91.136698,8.543264
3,136,138,2,-1.655629,-14.444938,0.564140
4,139,141,2,-3.767690,-32.327883,2.284442
...,...,...,...,...,...,...
126,1443,1446,3,0.734457,13.591784,2.785999
127,1447,1450,3,-3.670604,-68.426673,1.807789
128,1451,1457,6,-1.612545,-28.957327,6.025592
129,1458,1469,11,11.706939,206.837748,13.204336
